<a href="https://colab.research.google.com/github/juanibally/ARCHIVOS-DA/blob/main/PrimerInformeFinanciero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install yfinance pandas matplotlib fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=578e27374069fe0c8dcd0020ff6efa3827daf48ce619c766c0557616135bc20f
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [3]:
pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00


In [21]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from fpdf import FPDF
import os
import mplfinance as mpf
from google.colab import files

def calculate_rsi(df, period=14):
    """Calcula el RSI (Relative Strength Index)."""
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def calculate_macd(df, fast=12, slow=26, signal=9):
    """Calcula el MACD (Moving Average Convergence Divergence)."""
    df['EMA_fast'] = df['Close'].ewm(span=fast, adjust=False).mean()
    df['EMA_slow'] = df['Close'].ewm(span=slow, adjust=False).mean()
    df['MACD'] = df['EMA_fast'] - df['EMA_slow']
    df['Signal'] = df['MACD'].ewm(span=signal, adjust=False).mean()
    return df

def calculate_bollinger_bands(df, window=20, num_std_dev=2):
    """Calcula las Bandas de Bollinger."""
    df['MA20'] = df['Close'].rolling(window=window).mean()
    df['Upper_Band'] = df['MA20'] + (df['Close'].rolling(window=window).std() * num_std_dev)
    df['Lower_Band'] = df['MA20'] - (df['Close'].rolling(window=window).std() * num_std_dev)
    return df

def calculate_adx(df, window=14):
    """Calcula el ADX (Average Directional Index)."""
    df['TR'] = pd.concat([df['High'] - df['Low'],
                          (df['High'] - df['Close'].shift()).abs(),
                          (df['Low'] - df['Close'].shift()).abs()], axis=1).max(axis=1)
    df['+DM'] = df['High'].diff()
    df['-DM'] = -df['Low'].diff()
    df['+DM'] = df['+DM'].where(df['+DM'] > df['-DM'], 0)
    df['-DM'] = df['-DM'].where(df['-DM'] > df['+DM'], 0)

    df['TR14'] = df['TR'].rolling(window=window).sum()
    df['+DI14'] = 100 * df['+DM'].rolling(window=window).sum() / df['TR14']
    df['-DI14'] = 100 * df['-DM'].rolling(window=window).sum() / df['TR14']

    df['ADX'] = 100 * abs(df['+DI14'] - df['-DI14']) / (df['+DI14'] + df['-DI14'])
    return df

def plot_candlestick(df, ticker):
    """Genera y guarda el gráfico de velas japonesas con volumen."""
    df_copy = df.copy()
    df_copy.index.name = 'Fecha'
    df_copy = df_copy[['Open', 'High', 'Low', 'Close', 'Volume']]

    # Definir color de fondo
    mc = mpf.make_marketcolors(up='green', down='red', edge='inherit', wick='inherit')
    s = mpf.make_mpf_style(base_mpf_style='charles', marketcolors=mc)

    # Gráfico de velas japonesas
    filename = f'{ticker}_candlestick.png'
    mpf.plot(df_copy, type='candle', style=s, title=f'{ticker} - Precio', ylabel='Precio', ylabel_lower='Volumen', volume=True, savefig=filename)
    return filename

def plot_rsi(df):
    """Genera y guarda el gráfico del RSI."""
    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['RSI'], label='RSI', color='orange')
    plt.axhline(70, color='red', linestyle='--', label='Sobrecompra (70)')
    plt.axhline(30, color='green', linestyle='--', label='Sobreventa (30)')
    plt.title('RSI - Relative Strength Index')
    plt.xlabel('Fecha')
    plt.ylabel('RSI')
    plt.legend()
    plt.grid()
    plt.gca().set_facecolor('lightgrey')
    filename = 'RSI.png'
    plt.savefig(filename)
    plt.close()
    return filename

def plot_macd(df):
    """Genera y guarda el gráfico del MACD."""
    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['MACD'], label='MACD', color='blue')
    plt.plot(df.index, df['Signal'], label='Signal Line', color='red', linestyle='--')
    plt.title('MACD - Moving Average Convergence Divergence')
    plt.xlabel('Fecha')
    plt.ylabel('MACD')
    plt.legend()
    plt.grid()
    plt.gca().set_facecolor('lightgrey')
    filename = 'MACD.png'
    plt.savefig(filename)
    plt.close()
    return filename

def get_company_info(tickers):
    """Obtiene información adicional sobre las empresas."""
    company_info = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        info = stock.info
        company_info[ticker] = {
            'Nombre': info.get('longName', 'N/A'),
            'Sector': info.get('sector', 'N/A'),
            'Industria': info.get('industry', 'N/A'),
            'Descripción': info.get('longBusinessSummary', 'N/A'),
            'P/E': info.get('trailingPE', 'N/A'),
            'P/B': info.get('priceToBook', 'N/A'),
            'ROE': info.get('returnOnEquity', 'N/A'),
            'ROA': info.get('returnOnAssets', 'N/A'),
            'Dividendos': info.get('dividendRate', 'N/A'),
            'Rendimiento': info.get('dividendYield', 'N/A')
        }
    return company_info

def generate_pdf_report(tickers):
    """Genera un informe en PDF con la información y gráficos."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    for ticker in tickers:
        pdf.add_page()
        pdf.set_font('Arial', 'B', 16)
        pdf.cell(200, 10, f'Informe Financiero: {ticker}', ln=True, align='C')

        # Obtener datos de la empresa
        info = get_company_info([ticker])[ticker]
        pdf.set_font('Arial', '', 12)
        pdf.ln(10)
        pdf.cell(200, 10, f"Nombre: {info['Nombre']}", ln=True)
        pdf.cell(200, 10, f"Sector: {info['Sector']}", ln=True)
        pdf.cell(200, 10, f"Industria: {info['Industria']}", ln=True)
        pdf.multi_cell(0, 10, f"Descripción: {info['Descripción']}")
        pdf.cell(200, 10, f"P/E: {info['P/E']}", ln=True)
        pdf.cell(200, 10, f"P/B: {info['P/B']}", ln=True)
        pdf.cell(200, 10, f"ROE: {info['ROE']}", ln=True)
        pdf.cell(200, 10, f"ROA: {info['ROA']}", ln=True)
        pdf.cell(200, 10, f"Dividendos: {info['Dividendos']}", ln=True)
        pdf.cell(200, 10, f"Rendimiento: {info['Rendimiento']}", ln=True)

        # Obtener datos históricos y análisis técnico
        stock = yf.Ticker(ticker)
        df = stock.history(period='6mo', interval='1d')
        df = calculate_rsi(df)
        df = calculate_macd(df)
        df = calculate_bollinger_bands(df)
        df = calculate_adx(df)

        # Agregar gráfico de velas japonesas
        img_file = plot_candlestick(df, ticker)
        pdf.ln(10)
        pdf.image(img_file, x=15, w=180)
        os.remove(img_file)  # Borrar la imagen temporal

        # Agregar gráfico de RSI
        img_rsi = plot_rsi(df)
        pdf.ln(10)
        pdf.image(img_rsi, x=15, w=180)
        os.remove(img_rsi)  # Borrar la imagen temporal

        # Agregar gráfico de MACD
        img_macd = plot_macd(df)
        pdf.ln(10)
        pdf.image(img_macd, x=15, w=180)
        os.remove(img_macd)  # Borrar la imagen temporal

 # Guardar el archivo PDF en el directorio actual de Google Colab
    pdf_output = "/content/informe_financiero.pdf"
    pdf.output(pdf_output)

    # Descargar el archivo PDF
    files.download(pdf_output)

# Lista de empresas a analizar
tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NU', 'AMZN', 'NVDA', 'MELI', 'PAM', 'YPF', 'VIST']

# Llamar a la función para generar el informe PDF
generate_pdf_report(tickers)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>